# Audacityを使用して録音した音声ファイルmp3をWhisper + GPTで議事録書き出しとサマリ生成の実施

実験に使用している動画:

[【神々に愛された英雄】アルジュナの生い立ちとその時代／FGO×ゲームさんぽ#03](https://www.youtube.com/watch?v=VCbPdoaKTRo&t=1215s)(27:53)

In [1]:
!python3 -m pip install --upgrade pip

In [2]:
# !pip install openai==0.27.8
!pip install openai==1.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.0.0
    Uninstalling anyio-4.0.0:
      Successfully uninstalled anyio-4.0.0


In [3]:
!pip install python-dotenv tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.9 MB/s eta 0:00:0000:0100:01


In [4]:
# Release 20231117
!pip install git+https://github.com/openai/whisper.git@v20231117
# !pip install git+https://github.com/openai/whisper.git
#  (from sympy->torch->openai-whisper==20231117) (1.3.0)

  Cloning https://github.com/openai/whisper.git (to revision v20231117) to /tmp/pip-req-build-zupnis8i
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-zupnis8i
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=bd88758e6f65fc841c40b679d8746aef42d1615922ab54516cdf0ca8079473ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-xrybzu06/wheels/82/ef/3d/4c2e010696be8ff45a064a7629234956fc2c50b05bb9299d53
Successfully built openai-whisper


In [5]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [6]:
from tqdm import tqdm
from pathlib import Path

In [7]:
audio_file_path = Path('./data/fgo.mp3')

In [8]:
# 入力音声の変換
import soundfile as sf
import librosa

y, sr = librosa.load(audio_file_path)
# リサンプリング
y_16k = librosa.resample(y, sr, 16000)
n_samples = int(15 * 60 * 16000)

# 音声ファイルを15分ごとに分割する
segments = [y_16k[i:i+n_samples] for i in range(0, len(y_16k), n_samples)]

# 分割した音声ファイルを保存する
with Timer(prefix=f'audio segmentation'):
    for i, segment in tqdm(enumerate(segments)):
        sf.write(f"./data/meeting_{i}.wav", segment, 16000, format="WAV")

/tmp/ipykernel_43/3366858311.py:7: FutureWarning: Pass orig_sr=22050, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  y_16k = librosa.resample(y, sr, 16000)
2it [00:00,  3.02it/s]

audio segmentation 0.666[s]


In [9]:
# 音声処理: whisper
import whisper
import torch
import os

# 指定されたファイルの音声を文字起こしする関数
def generate_transcribe(file_path):
    # モデルの保存先を指定
    model_dir = Path('./model')
    # モデルの保存先指定
    os.environ['WHISPER_CACHE_DIR'] = str(model_dir)

    # モデル保存先のディレクトリが存在しない場合は作成
    model_dir.mkdir(parents=True, exist_ok=True)
    
    # GPUが利用可能か確認
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    # Whisper高速化テクニック
    # https://qiita.com/halhorn/items/d2672eee452ba5eb6241
    model = whisper.load_model('large', device=device)
    # モデルの処理を半精度浮動小数点数に設定して高速化
    _ = model.half()
    # モデルをCUDA（GPU）に移動
    # _ = model.cuda()

    # exception without following code
    # reason : model.py -> line 31 -> super().forward(x.float()).type(x.dtype)
    # Whisperモデル内のLayerNorm層を単精度浮動小数点数に設定
    # これを行わないと、型の不一致によりエラーが発生する
    for m in model.modules():
        if isinstance(m, whisper.model.LayerNorm):
            m.float()
    # 音声ファイルを文字起こし
    result = model.transcribe(file_path)
    return result

In [10]:
# トランスクリプト（文字起こしテキスト）を初期化
transcript = ""
# 各セグメントを文字起こしする
with Timer(prefix=f'generate transcription'):
    for i in range(len(segments)):
        # 各セグメントのファイルパスを生成
        file_path = f'./data/meeting_{i}.wav'
        # 文字起こしを実行
        transcribe = generate_transcribe(file_path)
        # 文字起こし結果の各セグメントからテキストを抽出し、トランスクリプトに追加
        for seg in transcribe['segments']:
            transcript += seg['text'] + "\n"

100%|█████████████████████████████████████| 2.88G/2.88G [47:58<00:00, 1.07MiB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


generate transcription 3159.078[s]


In [11]:
import openai
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()


def completion(text):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",  # モデルの指定
        messages=[
            {"role": "system", "content": "You summarize the transcript of the meeting."},
            {"role": "system", "content": "Outputs should be generated in step by step."},
            {"role": "user", "content": f"{text}"}
        ],
        temperature=0.8,
    )
    return response.choices[0].message.content

In [12]:
import json
def create_prompt(transcript):
    return f"""以下は、ある会議の書き起こしです。

{transcript}

この会議のサマリーを作成してください。サマリーは、以下のような形式で書いてください。

- 会議の目的
- 会議の内容
- 会議の結果

サマリー:
"""

In [13]:
import json
def create_prompt_act(transcript):
    return f"""以下は、ある会議の書き起こしです。

{transcript}

この会議の次にするアクションを作成してください。アクションの記述は以下のルールに従ってください。

・リスト形式で出力する (先頭は - を使う)
・簡潔に表現する

アクション:
"""

In [14]:
with Timer(prefix=f'create_summary'):
    prompt = create_prompt(transcript)
    summary = completion(prompt)

create_summary 31.651[s]


In [15]:
with Timer(prefix=f'create_action'):
    act_prompt = create_prompt_act(transcript)
    action = completion(act_prompt)

create_action 31.507[s]


In [16]:
print(transcript)

じゃあ彼らに続きまして
アルジナの方も見ていきたいと思います
お願いします
はい
召喚の方から一旦流させていただきます
はい
サーバント
アーチャー
アルジナと申します
マスター
私を存分にお使いください
はい
という感じで
こちらもちょっと言葉は少ないですけども
だいぶなんかいい人感というか
精錬な感じはやっぱりします
真面目そうなね
はい
マテリアルの方でも
至って勤勉
精錬
光明
盛大
火の打ち所のない完璧な人格
みたいな風な説明がされてますね
沖田さんとかアルジナとか
今回FGOで見て
どういうところとか注目しますか
肌の色が
黒いのが
アルジナは色黒だっていう接点
原点ではなんでいるので
そこを多く見とって
描かれているのかなっていうのと
黒っていうのは
マハーバーラタの中で
すごく重要な意味を持っていて
クリシュナで黒っていう意味なんですけど
マハーバーラタにクリシュナが3人出てきて
一人がいわゆるクリシュナ神のクリシュナですけど
もう一人がマハーバーラタの作者とされる
ビアサセンがクリシュナっていう名前もあるんですね
もう一人がドラウパディの別名が
クリシュナーって言うんですけど
最後の語尾を伸ばして女性系にするんですが
3人クリシュナがいるって言うんで
黒の女子子って言ってもいいようなところがあって
アルジナもクリシュナと二人で
クリシュナウって呼ばれて
二人のクリシュナって呼ばれるぐらいだから
黒っていうのがやっぱり重要なところかなって
思いました
そういう意味でやっぱり
全体的に黒い感じがあると
天竺さんとかどうですか
もうねやっぱりね
顔がいいっていう
2回目
こういう言い方あんまりしたくないんですけど
でも本当に顔がいい
スタイリングで着込んでいくじゃないですか
これが今第一なんですが
第二とかだと
そう服が増えるのがね最高ですね
増えるのが最高
ね最高です
第三とかになると
うんそう何度ついちゃうんですよ
かっこいいみたいな
頭像的な話で気になるところとかありますか
やっぱり持ってる弓と矢ですよね
それはさすがアルジナだな
弓といえばやっぱりアルジナ
アルジナですよね
ガンディーバですよね
そうそう
あれ又貸しなんですよ
また貸し
あのねアグニシンがアルジナに与えるんだけど
本来の持ち主はバルナシンって言うんで
そう海の神様
海の神様
神様

In [17]:
from IPython.display import display, Markdown, Latex

display(Markdown("## Summary\n\n" + summary + "\n\n## Action\n\n" + action))

## Summary

会議の目的:
この会議は、FGO（Fate/Grand Order）というゲームに登場するキャラクター「アルジュナ」についての深い理解と、その背景にあるインド神話の知識を共有することを目的としています。

会議の内容:
- アルジュナのキャラクター紹介と彼のサーヴァントとしての役割の説明。
- アルジュナの人物像とマテリアル（ゲーム内の設定資料）に記述されている性格、外見、武器に関する情報の共有。
- アルジュナの肌の色の意味と、インド神話「マハーバーラタ」における黒い肌の重要性についての論議。
- インド神話におけるアルジュナのエピソードや、彼の神々との関係、特にクリシュナとの関係についての議論。
- アルジュナと対照的なキャラクターであるカルナについての比較と、両者の関係性の検討。
- アルジュナの武器である「パーシュパタ」などの神話上の武器の説明とゲーム内での表現についての考察。
- ゲーム内でのアルジュナのプロフィールや彼のストーリー上の役割に関する詳細な読み解き。

会議の結果:
- アルジュナに関する詳細な情報が共有され、参加者間での理解が深まった。
- ゲームとインド神話との関連性が議論され、キャラクターの背景が明らかになった。
- アルジュナが「授かりの英雄」として、彼の神々との結びつきが非常に強いことが強調された。
- アルジュナとカルナの対立は、ゲーム内外で重要なテーマであり、その対比が詳細に分析された。

## Action

- 神話学の専門家にアルジュナの背景やエピソードに関する詳細な分析や解説を求める。
- ゲームのシナリオライターと協力して、アルジュナとカルナの関係性をより深く探るストーリー展開を考案する。
- アルジュナが使う武器やその背景に関する追加リサーチを行い、正確な情報を得る。
- マハーバーラタにおける他のキャラクター、例えばハヌマンやビーマについても研究し、関連するエピソードを紹介する。
- ゲーム内でアルジュナの旗印やモチーフとなるサル（ハヌマン）についての説明やエピソードを追加する。
- アルジュナのプロフィール、特に『授かりの英雄』としての部分をさらに詳細に展開する。
- アルジュナの道徳的ジレンマや彼が直面した試練について、より深く掘り下げる。
- ゲームのビジュアルデザインチームと連携し、武器や衣装の細部にまで神話の要素を反映させる。
- プレイヤーがアルジュナのバックストーリーや神話への理解を深められるような追加コンテンツを作成する。